In [41]:
import os
import random
import tensorflow as tf
import wandb
from wandb.integration.keras import WandbMetricsLogger

from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, ReLU, GlobalAveragePooling2D, Dropout

In [42]:
gpus = tf.config.list_physical_devices('GPU', )
print("GPUs available:", gpus)

GPUs available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [43]:
wandb.login()

True

In [44]:
def read_image_dataset(filename, validation_split=None, image_size=(256, 256), batch_size=32):
    train, validation = tf.keras.utils.image_dataset_from_directory(
        filename,
        validation_split=validation_split,
        subset="both" if validation_split else None,
        image_size=image_size,
        batch_size=batch_size,
        shuffle=True,
        seed=random.randint(1, 100),
    )

    return train, validation

In [55]:
def build_model(
    input_shape: tuple,
    num_classes,
    num_layers,
    filters_per_layer,
    kernel_size,
    dropout_rate,
):
    """
    Builds a generic, parameterized 1D CNN model.

    Args:
        input_shape (tuple): Shape of input data (timesteps, features).
        num_layers (int): Number of convolutional layers.
        num_classes (int): Number of output classes.
        filters_per_layer (list): Number of filters for each conv layer.
        kernel_size (list): Kernel size for each conv layer.
        dropout_rate (float): Dropout rate.
    Returns:
        keras.Model: Compiled Keras 1D CNN model.
    """

    model = Sequential(name="Generic2DCNN")
    model.add(Input(shape=input_shape))

    # Convolutional layers
    for i, no_filters in enumerate(filters_per_layer):
        for j in range(num_layers):
            model.add(Conv2D(
                filters=no_filters,
                kernel_size=kernel_size,
                padding='same',
                name=f'conv_{i + 1}_{j + 1}'
            ))
            model.add(ReLU(name=f'relu_{i + 1}_{j + 1}'))

        model.add(BatchNormalization(name=f'bn_{i + 1}'))
        model.add(ReLU(name=f'relu_{i + 1}'))
        model.add(Dropout(rate=dropout_rate))

    model.add(GlobalAveragePooling2D())

    # Dense head
    model.add(Dense(num_classes, activation='softmax', name='dense_1'))

    # Compile
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy'],
    )

    return model

In [46]:
def train_model(cfg):

    train, validation = read_image_dataset(
        'sample_data', validation_split=0.2, batch_size=cfg['batch_size']
    )

    input_shape = train.element_spec[0].shape[1:]  # (height, width, channels)
    num_classes = len(train.class_names)

    train = train.prefetch(tf.data.AUTOTUNE)

    m = build_model(
        input_shape=input_shape,
        num_classes=num_classes,
        num_layers=cfg['num_layers'],
        filters_per_layer=cfg['filters_per_layer'],
        kernel_size=cfg['kernel_size'],
        dropout_rate=cfg['dropout_rate']
    )

    best_model_file_path = os.path.join('trained_model', 'best_model.keras')
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            best_model_file_path, save_best_only=True, monitor="val_loss"
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", factor=0.5, patience=100, min_lr=0.0001
        ),
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=cfg['early_stop_patience'], verbose=1),
        WandbMetricsLogger()
    ]

    m.fit(
        train,
        validation_data=validation,
        batch_size=cfg['batch_size'],
        epochs=cfg['epochs'],
        callbacks=callbacks,
        verbose=1,
    )


In [47]:
sweep_config = {
    'method': 'bayes',  # or 'grid', 'random'
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'image_size': {'values': [(256,256)]},
        'num_layers': {'values': [1, 2]},
        'filters_per_layer': {'values': [[32, 64], [64, 128], [32, 64, 128]]},
        'kernel_size': {'values': [3, 5]},
        'batch_size': {'values': [16, 32]},
        'epochs': {'value': 1000},
        'early_stop_patience': {'values': [100]},
        'dropout_rate': {'values': [0.3, 0.5]}
    }
}


sweep_id = wandb.sweep(sweep_config, project="UC1-sweep")

Create sweep with ID: yv71mj90
Sweep URL: https://wandb.ai/michal-skalican-ui-sav/UC1-sweep/sweeps/yv71mj90


In [48]:
sweep_id

'yv71mj90'

In [49]:
def train_with_wandb():
    wandb.init()

    try:
        train_model(wandb.config)

    except (tf.errors.ResourceExhaustedError, RuntimeError) as e:
        if "out of memory" in str(e).lower():
            print(f"OOM detected for config: {wandb.config}")
        else:
            raise e
    finally:
        wandb.finish()

In [ ]:
wandb.agent('y5oui2gy', function=train_with_wandb, count=20)

wandb: Agent Starting Run: f4jhijdj with config:
wandb: 	batch_size: 32
wandb: 	dropout_rate: 0.5
wandb: 	early_stop_patience: 100
wandb: 	epochs: 1000
wandb: 	filters_per_layer: [32, 64, 128]
wandb: 	image_size: [256, 256]
wandb: 	kernel_size: 3
wandb: 	num_layers: 1


wandb: 
wandb: 🚀 View run eager-sweep-13 at: https://wandb.ai/michal-skalican-ui-sav/UC1-sweep/runs/xhcbzizv
wandb: Find logs at: wandb/run-20251107_122232-xhcbzizv/logs


/home/michal-skalican/Projects/SPICE/.venv/lib/python3.10/site-packages/pydantic/main.py:308: UserWarning: Pydantic serializer warnings:
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


Found 1080 files belonging to 2 classes.
Using 864 files for training.
Using 216 files for validation.
Epoch 1/1000
 8/27 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - loss: 0.7299 - sparse_categorical_accuracy: 0.5178